# Introduction
- A Keras model consists of multiple components.
    - An architecture, or configuration, which specifies what layers the model contains and how they are connected
    - A set of weights values (i.e. the state of the model)
    - An optimizer (defined by compiling the model)
    - A set of losses and metrics (defined by compiling the model or calling `add_loss()` or `add_metric()`).
- The Keras API makes it possible to save all of these pieces to disk at once, or to only selectively save some of them.
    - The standard practice is to save everything into a single archive in the TensorFlow SavedModel format.
    - Saving the architecture/configuration only, typically as a JSON file
    - Saving the weights values only, usually used when training the model.

# The short answer to saving & loading
- If you only have 10 seconds to read this guide, here's what you need to know.
- To save a Keras model:

In [1]:
# model = ...
# model.save('path/to/location')

- To load the model back:

In [2]:
from tensorflow import keras

In [3]:
# model = keras.models.load_model('path/to/location')

# Setup

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Whole-model saving & loading
- You can save an entire model to a single artifact. 
- It will include:
    - The model's architecture/config
    - The model's weight values (which were learned during training)
    - The model's compilation information (if `compile()`) was called
    - The optimizer and its state, if any (this enables you to restart training where you left)
- **APIs**
    - `model.save()` or `tf.keras.models.save_model()`
    - `tf.keras.models.load_model()`
- There are two formats you can use to save an entire model to disk: the **TensorFlow SavedModel** format, and the older **Keras H5** format. 
    - The recommended format is SavedModel. 
        - It is the default when you use `model.save()`.
    - You can switch to the H5 format by:
        - Passing `format='h5'` to `save()`.
        - Passing a filename that ends in `.h5` or `.keras` to `save()`.

## SavedModel format

In [5]:
def get_model():
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [6]:
model = get_model()

In [7]:
# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

Train on 128 samples
128/128 [==============================] - 1s 5ms/sample - loss: 0.6599


In [8]:
# Calling model.save(.../my_model) creates a SavedModel folder "my_model"
model.save("/Users/yuangchen/Documents/Python/TensorFlow Core/TensorFlow Core 05 - Saved files/my_model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /Users/yuangchen/Documents/Python/TensorFlow Core/TensorFlow Core 05 - Saved files/my_model/assets


In [9]:
reconstructed_model = keras.models.load_model("/Users/yuangchen/Documents/Python/TensorFlow Core/TensorFlow Core 05 - Saved files/my_model")

In [10]:
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

In [11]:
# The reconstructed model is already compiled and has retained the optimizer state
# So we can resume training
reconstructed_model.fit(test_input, test_target)

Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.5783


### What the SavedModel contains
- Calling `model.save()` creates a folder that contains the following files.

In [12]:
ls "/Users/yuangchen/Documents/Python/TensorFlow Core/TensorFlow Core 05 - Saved files/my_model"

assets/         saved_model.pb  variables/


- The model architecture, and training configuration (including the optimizer, losses, and metrics) are stored in `saved_model.pb`. 
- The weights are saved in the `variables/` directory.

## How SavedModel handles custom objects
- When saving the model and its layers, the SavedModel format stores the class name, call function, losses, and weights (and the config, if implemented).
    - The call function defines the computation graph of the model/layer.
    - In the absence of the model/layer config, the call function is used to create a model that exists like the original model which can be trained, evaluated, and used for inference.
- Nevertheless, it is always a good practice to define the `get_config` and `from_config` methods when writing a custom model or layer class. 
    - This allows you to easily update the computation later if needed. 
    - See the section about Custom objects for more information.
- Below is an example of what happens when loading custom layers from he SavedModel format without overwriting the config methods.

In [13]:
class CustomModel(keras.Model):
    def __init__(self, hidden_units):
        super().__init__()
        self.dense_layers = [keras.layers.Dense(u) for u in hidden_units]

    def call(self, inputs):
        X = inputs
        for layer in self.dense_layers:
            X = layer(X)
        return X

In [14]:
model = CustomModel([16, 16, 10])

In [15]:
# # Build the model by calling it
# input_arr = tf.random.uniform((1, 5))
# outputs = model(input_arr)
# model.save("/Users/yuangchen/Documents/Python/TensorFlow Core/TensorFlow Core 05 - Saved files/my_model")

# del CustomModel

# loaded = keras.models.load_model("/Users/yuangchen/Documents/Python/TensorFlow Core/TensorFlow Core 05 - Saved files/my_model")
# np.testing.assert_allclose(loaded(input_arr), outputs)

## Keras H5 format
- Keras also supports saving a HDF5 file contaning the model's architecure, weights values, and `compile()` information.
- It is a light-weight alternative to SaveModel.

In [16]:
model = get_model()

In [17]:
# Train the model
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 0.3542


In [18]:
# Save the model as a H5 file
model.save("/Users/yuangchen/Documents/Python/TensorFlow Core/TensorFlow Core 05 - Saved files/my_h5_model.h5")

In [19]:
# Reconstruct the model using the H5 file
reconstructed_model = keras.models.load_model("/Users/yuangchen/Documents/Python/TensorFlow Core/TensorFlow Core 05 - Saved files/my_h5_model.h5")

In [20]:
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

In [21]:
# The reconstructed model is already compiled and has retained the optimizer state, so training can resume
reconstructed_model.fit(test_input, test_target)

Train on 128 samples
128/128 [==============================] - 0s 1ms/sample - loss: 0.3150


- Compared to the SavedModel format, there are two things that don't get included in the H5 file.
    - **External losses & metrics** added via `model.add_loss()` & `model.add_metric()` are not saved (unlike SavedModel). 
        - If you have such losses & metrics on your model and you want to resume training, you need to add these losses back yourself after loading the model. 
        - Note that this does not apply to losses/metrics created inside layers via `self.add_loss()` & `self.add_metric()`. 
        - As long as the layer gets loaded, these losses & metrics are kept, since they are part of the `call` method of the layer.
    - The **computation graph of custom objects** such as custom layers is not included in the saved file. 
        - At loading time, Keras will need access to the Python classes/functions of these objects in order to reconstruct the model.

# Saving the architecture
- The model's configuration (or architecture) specifies what layers the model contains, and how these layers are connected.
    - - Note this only applies to models defined using the functional or Sequential apis not subclassed models.
- If you have the configuration of a model, then the model can be created with a freshly initialized state for the weights and no compilation information.

## Configuration of a Sequential model or Functional API model
- These types of models are explicit graphs of layers, and their configuration is always available in a structured form.
- APIs:
    - **`get_config()`** and **`from_config`**
        - Calling `config = model.get_config()` will return a Python dict containing the configuration of the model. 
        - The same model can then be reconstructed via `Sequential.from_config(config)` (for a Sequential model) or `Model.from_config(config)` (for a Functional API model).
        - The same workflow also works for any serializable layer.

In [22]:
# Layer example
layer = keras.layers.Dense(3, activation='relu')
layer_config = layer.get_config()
new_layer = keras.layers.Dense.from_config(layer_config)

In [23]:
# Sequential model example
model = keras.Sequential([
    keras.Input((32,)),
    keras.layers.Dense(1)
])

config = model.get_config()

new_model = keras.Sequential.from_config(config)

In [24]:
# Functional model example
inputs = keras.Input((32,))
outputs = keras.layers.Dense(1)(inputs)

model = keras.Model(inputs, outputs)

config = model.get_config()
new_model = keras.Model.from_config(config)

- APIs (cont'd)
    - **`tf.keras.models.to_json()`** and **`tf.keras.models.model_from_json()`** are similar to `get_config()` and `from_config()`, except it turns the model into a JSON string.
        - The JSON string can then be loaded without the original model class.
        - It is also specific to models, not for layers.

In [25]:
model = keras.Sequential([
    keras.Input((32,)),
    keras.layers.Dense(1)
])

json_config = model.to_json()

new_model = keras.models.model_from_json(json_config)

## Custom objects
### (Subclassed) Model and layers
- The architecture of subclassed models and layers are defined in the methods `__init__` and `call`. 
    - They are considered Python bytecode, which cannot be serialized into a JSON-compatible config -- you could try serializing the bytecode (e.g. via `pickle`), but it's completely unsafe and means your model cannot be loaded on a different system.
- In order to save/load a model with custom-defined layers, or a subclassed model, you should **overwrite the `get_config` and optionally `from_config` methods**. 
    - `get_config` should return a JSON-serializable dictionary in order to be compatible with the Keras architecture- and model-saving APIs.
    - `from_config(config) (classmethod)` should return a new layer or model object that is created from the config. 
        - The default implementation returns `cls(**config)`.

In [26]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, a):
        self.var = tf.Variable(a, name='var_a')
        
    def call(self, inputs, training=False):
        if training:
            return inputs * self.var
        else:
            return inputs
        
    def get_config(self):
        return {'a': self.var.numpy()}
    
    # There's actually no need to define `from_config` here, since returning `cls(**config)` is the default behavior.
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [27]:
layer = CustomLayer(5)
layer.var.assign(2)

<tf.Variable 'UnreadVariable' shape=() dtype=int32, numpy=2>

In [28]:
serialized_layer = keras.layers.serialize(layer)
new_layer = keras.layers.deserialize(
    serialized_layer, custom_objects={"CustomLayer": CustomLayer}
)

- Additionally, you should use **register the custom object** so that Keras is aware of it.
    - Keras keeps a note of which class generated the config.
    - From the example above, `tf.keras.layers.serialize` generates a serialized form of the custom layer.

In [29]:
{'class_name': 'CustomLayer', 'config': {'a': 2} }

{'class_name': 'CustomLayer', 'config': {'a': 2}}

- Keras keeps a master list of all built-in layer, model, optimizer, and metric classes, which is used to find the correct class to call `from_config`. 
    - If the class can't be found, than an error is raised (`Value Error: Unknown layer`). 
    - There are a few ways to register custom classes to this list:
    - Setting `custom_objects` argument in the loading function. (see the example in section above "Defining the config methods")
    - `tf.keras.utils.custom_object_scope` or `tf.keras.utils.CustomObjectScope`
    - `tf.keras.utils.register_keras_serializable`

### Custom functions
- Custom-defined functions (e.g. activation loss or initialization) do not need a `get_config` method. 
- The function name is sufficient for loading as long as it is registered as a custom object.

In [30]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='random_normal',
            trainable=True 
        )
        
        self.b = self.add_weight(
            shape=(self.units, ),
            initializer='random_normal',
            trainable=True
        )
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):
        config = super().get_config()
        return {**config, "units": self.units}

In [31]:
def custom_activation(X):
    return tf.nn.tanh(X) ** 2

In [32]:
# Make a model with the CustomLayer and custom_activation
inputs = keras.Input((32,))
X = CustomLayer(32)(inputs)
outputs = keras.layers.Activation(custom_activation)(X)

model = keras.Model(inputs, outputs)

In [33]:
# Retrieve the config
config = model.get_config()

In [34]:
# At loading time, register the custom object with a 'custom_object_scope'
custom_objects = {"CustomLayer": CustomLayer, "custom_activation": custom_activation}

with keras.utils.custom_object_scope(custom_objects):
    new_model = keras.Model.from_config(config)

## In-memory model cloning
- You can also do in-memory cloning of a model via `tf.keras.models.clone_model()`.
- This is equivalent to geting the config then recreating the model from its config.
    - Thus, it does not preserve compilation information or layer weights values.

In [35]:
with keras.utils.custom_object_scope(custom_objects):
    new_model = keras.models.clone_model(model)

# Saving & loading only the model's weights values
- You can choose to only save & load a model's weights. 
- This can be useful if:
    - You only need the model for **inference**: in this case you won't need to restart training, so you don't need the compilation information or optimizer state.
    - You are doing **transfer learning**: in this case you will be training a new model reusing the state of a prior model, so you don't need the compilation information of the prior model.
    
## APIs for in-memory weight transfer
- Weights can be copied between different objects by using **`get_weights`** and **`set_weights`**.
    - `tf.keras.layers.Layer.get_weights()` returns a list of numpy arrays.
    - `tf.keras.layers.Layer.set_weights()` sets the model weights to the values in the `weights` argument.
- Example: Transferring weights from one model to another with a compatible architecture, in memory.

In [36]:
# Create a simple functional model
inputs = keras.Input(shape=(784,), name='digits')
X = keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
X = keras.layers.Dense(64, activation='relu', name='dense_2')(X)
outputs = keras.layers.Dense(10, name='predictions')(X)
functional_model = keras.Model(inputs, outputs, name='3_layer_mlp')

In [37]:
# Define a subclassed model with the same architecture
class SubclassedModel(keras.Model):
    def __init__(self, output_dim, name=None):
        super().__init__(name=name)
        self.output_dim = output_dim
        self.dense_1 = keras.layers.Dense(64, activation="relu", name="dense_1")
        self.dense_2 = keras.layers.Dense(64, activation="relu", name="dense_2")
        self.dense_3 = keras.layers.Dense(output_dim, name="predictions")
        
    def call(self, inputs):
        X = self.dense_1(inputs)
        X = self.dense_2(X)
        X = self.dense_3(X)
        return X
    
    def get_config(self):
        return {"output_dim": self.output_dim, "name": self.name}

In [38]:
subclassed_model = SubclassedModel(10)

In [39]:
# Call the subclassed model to create the weights
subclassed_model(tf.ones((1, 784)))

<tf.Tensor: id=3450, shape=(1, 10), dtype=float32, numpy=
array([[-2.4197953 , -0.23583509, -0.7261448 ,  0.58351696, -1.575793  ,
         1.4143335 , -1.3235005 , -0.550333  ,  2.0375602 ,  0.85234565]],
      dtype=float32)>

In [40]:
# Copy weights from functional_model to subclassed_model
subclassed_model.set_weights(functional_model.get_weights())

In [41]:
assert len(functional_model.weights) == len(subclassed_model.weights)
for a, b in zip(functional_model.weights, subclassed_model.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

- The case of stateless layers
    - Because stateless layers do not change the order or number of weights, models can have compatible architectures even if there are extra/missing stateless layers.

In [42]:
inputs = keras.Input(shape=(784,), name="digits")
X = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
X = keras.layers.Dense(64, activation="relu", name="dense_2")(X)
outputs = keras.layers.Dense(10, name="predictions")(X)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

In [43]:
inputs = keras.Input(shape=(784,), name="digits")
X = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
X = keras.layers.Dense(64, activation="relu", name="dense_2")(X)

# Add a dropout layer, which is stateless (i.e. doesn't contain any weights)
X = keras.layers.Dropout(0.5)(X)
outputs = keras.layers.Dense(10, name="predictions")(X)

functional_model_with_dropout = keras.Model(
    inputs=inputs, outputs=outputs, name="3_layer_mlp"
)

In [44]:
functional_model_with_dropout.set_weights(functional_model.get_weights())

## APIs for saving weights to disk & loading them back
- Weights can be saved to disk by calling `model.save_weights` in the following formats:
    - TensorFlow Checkpoint
    - HDF5
- The default format is TensorFlow Checkpoint, and there are two ways to specify the save format:
    - `save_format` argument: set `save_format="tf"` or `save_format="h5"`
    - `path` argument: if the path ends with `.h5` or `.hdf5`, then the HDF5 format is used; otherwise, it will be saved as a TensorFlow Checkpoint.
- More detailes available at https://www.tensorflow.org/guide/keras/save_and_serialize